# This is a script for managing initial conditions (ICs) for simulations in Arepo2.

In [ ]:
import os
cwd = os.getcwd()
print(cwd)

import numpy as np
import matplotlib.pyplot as plt
import h5py
from ic_tools import *
import shutil

# Set model parameters in yaml files, then

# Generate initial conditions with 'make_many_ics.sh'

In [ ]:
# set corresponding parameters for model sizes
# [V1-V4, V5-V8, V9-V12, V13-V16]
boxsizes = np.array([1000, 1500, 2000, 2000])
bh_masses = np.array([0.316, 1, 3.16, 10]) * 1e6  # in Msun
radii = np.array([20, 25, 30, 40]) # in kpc
heights = np.array([1, 1, 1, 1.5]) # in kpc

# [V1-2, V3-4, V5-6, V7-8, V9-10, V11-12, V13-14, V15-16]
resolutions = [600, 600, 2000, 2000, 3000, 6000, 10000, 20000] # Msun

# set paths
ic_path = '/cosma8/data/dp058/dc-faes1/initial_conditions/'
run_path = '/cosma8/data/dp058/dc-faes1/arepo2_base/'

In [ ]:
for n in np.arange(1, 17, 1):

    print(f"\nProcessing V{n}...")

    boxsize = boxsizes[(n-1) // 4]
    bh_mass = bh_masses[(n-1) // 4]
    radius = radii[(n-1) // 4]
    height = heights[(n-1) // 4]

    # fix initial condition file format
    fix_ics(os.path.join(ic_path, f"V{n}.hdf5"), boxsize=boxsize, unit_length=3.0856e20, unit_mass=1.991e33, unit_velocity=1.0e5)

    # move ics to run directory
    target_dir = os.path.join(run_path, f"snapshots_V{n}")
    src = os.path.join(ic_path, f"new_V{n}.hdf5")
    dst = os.path.join(target_dir, f"V{n}.hdf5")

    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    if os.path.exists(src):
        try:
            print(f"Moving {src} -> {dst}")
            shutil.move(src, dst)
        except Exception as e:
            print(f"Error moving {src} -> {dst}: {e}")
    else:
        print(f"Source file not found, skipping: {src}")

    add_background_grid(dst, disk_radius_in_kpc=radius, disk_height_in_kpc=height, ndensity_per_cm3=1e-12, grid_size=32, boxsize=boxsize, margin=20)

    add_bh_particle(dst, bh_mass)

    f = h5py.File(dst, 'r')
    coords = np.array(f['PartType0']['Coordinates'])
    f.close()

    # Plot XY and XZ planes
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.scatter(coords[:, 0], coords[:, 1], s=2, color="red")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(f"Particle positions (XY plane)")
    plt.axis("equal")

    plt.subplot(1, 2, 2)
    plt.scatter(coords[:, 0], coords[:, 2], s=2, color="blue")
    plt.xlabel("X")
    plt.ylabel("Z")
    plt.title(f"Particle positions (XZ plane)")
    plt.axis("equal")
    plt.show()


# Run AREPO to get snapshot 000, then

In [ ]:
for n in np.arange(1, 17, 1):

    print(f"\nProcessing V{n}_000...")

    boxsize = boxsizes[(n-1) // 4]
    bh_mass = bh_masses[(n-1) // 4]
    radius = radii[(n-1) // 4]
    height = heights[(n-1) // 4]

    filepath = os.path.join(run_path, f"snapshots_V{n}", f"V{n}_000.hdf5")

    # add additional properties to the ICs
    # (Arepo will skip B field on read if started from IC unless behavior has been changed in snap_io.cc (search for 'MHD_SEEDFIELD'))
    add_dust_temperature(filepath, dust_temperature_in_K=2.0)
    add_chem_abundances(filepath, chem_abundances='NL97')
    add_magnetic_field(filepath, B0_in_gauss=1e-12, direction='toroidal')

    check_ics(filepath, boxsize=boxsize, unit_length=3.0856e20, unit_mass=1.991e33, unit_velocity=1.0e5)

    print("\n Check sim properties...")
    sim_mass, sim_ndensity, sim_temp = get_sim_mass_ndensity_temp(filepath, radii_in_kpc=[0, 300], half_heights_in_kpc=[0, 200])

    print("\n Check disk properties...")
    disk_mass, disk_ndensity, disk_temp = get_disk_mass_ndensity_temp(filepath, radii_in_kpc=[0, radius], half_heights_in_kpc=[0, height])

    print("\n Check CGM properties...")
    # cgm_mass, cgm_ndensity, cgm_temp = get_cgm_mass_ndensity_temp(filepath, radii_in_kpc=[radius, 100], half_heights_in_kpc=[height, 100])
    set_CGM_ndensity(filepath, CGM_ndensity=1e-12, density_threshold_cgs=1, radii_in_kpc=[radius, 100], half_heights_in_kpc=[height, 100])
    set_CGM_temperature(filepath, CGM_temp_K=1e7, temp_threshold_K=1, radii_in_kpc=[radius, 100], half_heights_in_kpc=[height, 100])
    cgm_mass, cgm_ndensity, cgm_temp = get_cgm_mass_ndensity_temp(filepath, radii_in_kpc=[radius, 100], half_heights_in_kpc=[height, 100])

    # check magnetic field
    plot_field(filepath, plane='XY')
    plot_field(filepath, plane='XZ')

# run from snapshot 000 (using option 2)